# <font color='red'> Description du projet </font>

## <font color='blue'>Présentation du problème </font>

L’objectif de ce projet est d’estimer **les temps de réponse et de mobilisation** de la Brigade des Pompiers de Londres. La brigade des pompiers de Londres est le service d'incendie et de sauvetage le plus actif du Royaume-Uni  et l'une des plus grandes organisations de lutte contre l'incendie et de sauvetage au monde.

Le premier jeu de données fourni contient les détails de chaque incident traité depuis janvier 2009. Des informations sont fournies sur la date et le lieu de l'incident ainsi que sur le type d'incident traité. Il est composé de deux fichiers

*   LFB Incident data from 2009 - 2017.xlsx
*   LFB Incident data from 2018 onwards.csv

Le second fichier peut-être récupéré à l'aide du lien : 'https://data.london.gov.uk/download/london-fire-brigade-incident-records/f5066d66-c7a3-415f-9629-026fbda61822/LFB%20Incident%20data%20from%202018%20onwards.csv.xlsx' pour avoir la dernière version du fichier. En effet, les données sont mises à jour tous les mois. Il faut compter au moins 7 minutes pour la lecture des données.

<br>

Le second jeu de données contient les détails de chaque camion de pompiers envoyé sur les lieux d'un incident depuis janvier 2009. Des informations sont fournies sur l'appareil mobilisé, son lieu de déploiement et les heures d'arrivée sur les lieux de l'incident. Il est composé de trois fichiers

*   LFB Mobilisation data from January 2009 - 2014.xlsx
*   LFB Mobilisation data from 2015 - 2020.xlsx
*   LFB Mobilisation data from January 2009 - 2014.xlsx

Le dernier fichier peut-être récupéré à l'aide du lien : 'https://data.london.gov.uk/download/london-fire-brigade-mobilisation-records/3ff29fb5-3935-41b2-89f1-38571059237e/LFB%20Mobilisation%20data%202021%20-%202024.xlsx' pour avoir la dernière version du fichier (mise à jour mensuelle). Il faut compter environ 17 minutes pour la lecture des données.

## <font color='blue'> Etapes précédentes </font>

*   1 - Exploration des données : premières analyses, concaténation des différents fichiers puis jointure des 2 types de données (incident / mobilisation)
*   2 - Data visualisation.ipynb : visualisation des données, étude de la variable à prédire (temps de réponse total) en fonction des variables explicatives, création d'un jeu de données pour la modélisation
*   5 - Preprocessing final : création des jeux de données pour la modélisation faite dans ce notebook
*   6 - Modelisation Preproc final - Pred Continue : modélisation de la variable continue `TotalResponseTime_BC`
*  7 - Modelisation Preproc final - Pred 10Cat : modélisation de la variable continue `ResponseTimeCategory`



*Nota Bene* : Les notebooks numérotés 3 et 4 ont été conservés pour montrer un premier travail de modélisation qui a été abandonné par la suite (voir détails ci-dessous)


## <font color='blue'>Etapes dans ce notebook </font>

Dans ce notebook, nous avons modélisé le temps de réponse, après catégorisation en 6 classes (`ResponseTimeCategory2`). Nous avons testé plusieurs algorithmes de classification.

Pour évaluer nos modèles (et les comparer) nous utilisons les métriques suivantes :
- l'exactitude (accuracy) calculée sur la totalité des classes
- la précision médiane, minimale et maximale (cette mesure étant calculée pour chaque classe)
- le rappel (recall) médian, minimal et maximal
- le f1-score médian, minimal et médian
Ces mesures sont calculées sur le jeu de données de validation.

Les résultats étant mauvais - accuracy de l'ordre de 15% pour tous les modèles - nous avons peu commenté ce notebook.

# <font color='red'>1) Préparation de l'environement de travail </font>

## <font color='blue'>Installation des modules </font>

In [ ]:
#!pip install matplotlib
#!pip install Seaborn
#!pip install openpyxl
#!pip install scipy
#!pip install geopandas
#!pip install scikit-learn
#!pip install statsmodels
#!pip install folium
#!pip install plotly
#!pip install --upgrade seaborn
#!pip install jupyter
#!pip install nbformat
!pip install lightgbm
!pip install xgboost
#!pip optuna

## <font color='blue'>Importation des bibliothèques </font>

In [ ]:
import pandas as pd  #Pour les dataframe
import numpy as np #Pour le calcul numérique
import datetime as dt # Pour le calcul sur les dates

## Libraries pour les graphiques
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D # graphique 3D
import plotly.express as px  #graphique 3D dynamique
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.offline as offline

## Libraries pour statistiques
# régression linéaire
import statsmodels.api as sm
from statsmodels.formula.api import ols
# tests statistiques
from scipy.stats import shapiro , kstest # tests de sur la normalité de la distributin
from scipy.stats import bartlett # tests sur les variances
from scipy.stats import kruskal #  comparaison des médianes
from scipy.stats import spearmanr
from scipy.stats import loguniform, uniform
from scipy import stats # notamment pour boxplot
# metrics
from sklearn.metrics import f1_score, accuracy_score, make_scorer, classification_report, recall_score, precision_recall_fscore_support

# Libraries divers
from copy import deepcopy  # gestion des copies

# Pour la séparation du jeu de données
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score, cross_validate, RandomizedSearchCV, KFold
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LassoCV, ElasticNet, LogisticRegression

from sklearn.feature_selection import SelectKBest, f_regression

from sklearn.metrics import mean_squared_error, r2_score

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, plot_tree
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

from lightgbm import LGBMRegressor, LGBMClassifier
from xgboost import XGBRegressor, XGBClassifier
# import optuna


## <font color='blue'>Liaison avec le drive (pour travailler sur GoogleColab) </font>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# <font color='red'>2) Données pour la modélisation </font>

## <font color='blue'>2.a) Chargement </font>


Chargement des données depuis GoogleColab

In [ ]:
train = pd.read_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Complete/Train_Dataset.csv', low_memory=False)
validation = pd.read_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Complete/Validation_Dataset.csv', low_memory=False)

train_reduit = pd.read_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Reduit2/Train_Dataset.csv', low_memory=False)
validation_reduit = pd.read_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Reduit2/Validation_Dataset.csv', low_memory=False)

Chargement des données depuis un emplacement en local

In [ ]:
train = pd.read_csv('../Data/Datapreprocessing/Complete/Train_Dataset.csv', low_memory=False)
validation = pd.read_csv('../Data/Datapreprocessing/Complete/Validation_Dataset.csv', low_memory=False)

train_reduit = pd.read_csv('../Data/Datapreprocessing/Reduit2/Train_Dataset.csv', low_memory=False)
validation_reduit = pd.read_csv('../Data/Datapreprocessing/Reduit2/Validation_Dataset.csv', low_memory=False)

## <font color='blue'>2.b) Sélection des variables </font>

In [ ]:
# X : variable explicative du modele --> on supprime IncidentNumber et les 4 variables à prédire
# y : la variable à prédire est ResponseTimeCategory
X_train = train.drop(columns=['IncidentNumber','ResponseTimeCategory', 'ResponseTimeCategory2', 'ResponseTimeBinary', 'TotalResponseTime_BC'])
y_train = train[['ResponseTimeCategory2']]

X_val = validation.drop(columns=['IncidentNumber','ResponseTimeCategory', 'ResponseTimeCategory2', 'ResponseTimeBinary', 'TotalResponseTime_BC'])
y_val = validation[['ResponseTimeCategory']]


# même principe sur le jeux de données réduit (pour la recherche d'hyperparamètres)
X_train_r = train_reduit.drop(columns=['IncidentNumber','ResponseTimeCategory', 'ResponseTimeCategory2', 'ResponseTimeBinary', 'TotalResponseTime_BC'])
y_train_r = train_reduit[['ResponseTimeCategory2']]

X_val_r = validation_reduit.drop(columns=['IncidentNumber','ResponseTimeCategory', 'ResponseTimeCategory2', 'ResponseTimeBinary', 'TotalResponseTime_BC'])
y_val_r = validation_reduit[['ResponseTimeCategory']]


# <font color='red'> 3) Méthodes
 </font>







Dans la suite de ce notebook, nous utilisons les méthodes définies ici

In [ ]:
### evaluation du modèle
# cette méthode calcul les différentes métriques permettant l'évaluation du modèle
def evaluate_model(y_true, y_pred, model_name):
    print(f"\n{model_name}")
    precision, recall, fscore, _ = precision_recall_fscore_support(y_true, y_pred)
    print("Overall accuracy :", np.round(accuracy_score(y_true, y_pred),4))
    print("min precision: ", np.round(precision.min(),4))
    print("median precision : ", np.round(np.median(precision),4))
    print("max precision : ", np.round(precision.max(),4))
    print("min recall : ", np.round(recall.min(),4))
    print("median recall : ", np.round(np.median(recall),4))
    print("max recall : ", np.round(recall.max(),4))
    print("min fscore : ", np.round(fscore.min(),4))
    print("median fscore : ", np.round(np.median(fscore),4))
    print("max fscore : ", np.round(fscore.max(),4))


In [ ]:
### estimation des paramètres, prédiction et calcul des métriques d'évaluation
def train_and_evaluate_model(model, model_name, X_train, y_train, X_val, y_val):
    y_train = y_train.to_numpy().ravel() if isinstance(y_train, pd.DataFrame) else y_train
    y_val = y_val.to_numpy().ravel() if isinstance(y_val, pd.DataFrame) else y_val
    # Entraînement du modèle
    model.fit(X_train, y_train)

    # Prédiction sur l'ensemble de validation
    y_pred_val = model.predict(X_val)

    # Évaluation sur l'ensemble de validation
    evaluate_model(y_val, y_pred_val, model_name)

    return y_pred_val

In [ ]:
def find_best_params_and_predict(model, param_distributions, X_train, y_train, X_val, y_val, model_name, n_iter):
    """
    Effectue une recherche des meilleurs paramètres pour un modèle donné avec RandomizedSearchCV, et retourne les prédictions.
    """
    randomized_search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_distributions,
        n_iter=n_iter,               # Nombre d'itérations de recherche aléatoire
        scoring='accuracy',      # métrique à utiliser pour le choix d'hyper paramètres
        cv=3,                    # Nombre de sub dataset pour la validation croisée
        n_jobs=-1,               # Utilisation de tous les cœurs disponibles
        verbose=1,               # Affiche les logs de progression
        random_state=42          # Pour garantir la reproductibilité
    )

    # Exécute la recherche
    randomized_search.fit(X_train, y_train)

    # Meilleurs paramètres et score
    print(f"\n{model_name} - Best Parameters: {randomized_search.best_params_}")
    print(f"{model_name} - Best Cross-Validation Score: {randomized_search.best_score_:.4f}")

    # Meilleur modèle entraîné
    best_model = randomized_search.best_estimator_

    # Prédictions sur l'ensemble de validation
    y_pred = best_model.predict(X_val)

    # Évaluation
    evaluate_model(y_val, y_pred, model_name)

    return y_pred, randomized_search.best_params_


# <font color='red'> 4) Classification
 </font>


## <font color='blue'>4.a) Support Vector Machine (SVM) </font>

Malgré plusieurs essais et des exécutions de plusieurs heures, l'algorithme SVM n'a jamais convergé.

In [ ]:
svm_model = SVC(random_state=42, probability=True)
y_pred_svm = train_and_evaluate_model(svm_model, "SVM", X_train, y_train, X_val, y_val)

## <font color='blue'>4.b) K-plus proches voisins </font>

### Première estimation

In [ ]:
knn_model = KNeighborsClassifier()
y_pred_KNC=train_and_evaluate_model(knn_model, "k-Nearest Neighbors", X_train, y_train, X_val, y_val)


k-Nearest Neighbors
Overall accuracy : 0.1444
min precision:  0.0
median precision :  0.0733
max precision :  0.317
min recall :  0.0
median recall :  0.0727
max recall :  0.7249
min fscore :  0.0
median fscore :  0.0721
max fscore :  0.4411


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



### Recherche des hyper paramètres

La recherche d'hyper paramètres sur le jeu de données réduit améliore légèrement l'accuracy. Cependant, comme elle reste inférieure à 20%, nous n'avons pas appliqué ce modèle au jeu de données complet.

In [ ]:

knn_param_distributions = {
    'n_neighbors': [5, 10, 20, 50],
    'weights': ['uniform', 'distance'],
    'leaf_size': [1, 5, 15, 30],
    'p': [1, 2, 5]
}

knn_model_best = KNeighborsClassifier()

y_pred_Bestknn, best_params_knn= find_best_params_and_predict(knn_model_best, knn_param_distributions
                                                              , X_train_r, y_train_r, X_val_r, y_val_r
                                                              , "Optimized Random Forest Regressor", 50)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


C:\Users\ADUBOIS\AppData\Local\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().




Optimized Random Forest Regressor - Best Parameters: {'weights': 'uniform', 'p': 5, 'n_neighbors': 50, 'leaf_size': 30}
Optimized Random Forest Regressor - Best Cross-Validation Score: 0.3812

Optimized Random Forest Regressor
Overall accuracy : 0.1486
min precision:  0.0
median precision :  0.0693
max precision :  0.3319
min recall :  0.0
median recall :  0.0673
max recall :  0.6962
min fscore :  0.0
median fscore :  0.0622
max fscore :  0.4495


C:\Users\ADUBOIS\AppData\Local\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



## <font color='blue'>4.c) Arbre de décision </font>

In [ ]:
dt_model = DecisionTreeClassifier(random_state=42)
y_pred_DT= train_and_evaluate_model(dt_model, "Decision Tree", X_train, y_train, X_val, y_val)


Decision Tree
Overall accuracy : 0.1422
min precision:  0.0
median precision :  0.0826
max precision :  0.3205
min recall :  0.0
median recall :  0.092
max recall :  0.6478
min fscore :  0.0
median fscore :  0.0865
max fscore :  0.4289


C:\Users\ADUBOIS\AppData\Local\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [ ]:
feat_imp = pd.DataFrame({'importance': dt_model.feature_importances_}, index=dt_model.feature_names_in_)
feat_imp.sort_values(by=['importance'], ascending=False).head(10)

,importance
distStd,0.732260
ratioStd,0.041472
H1117,0.030344
Bor_inc_rep,0.020095
PropCat_Non Residential,0.012927
PropCat_Dwelling,0.011812
PropCat_Road Vehicle,0.011151
PropCat_Outdoor Structure,0.010520
H26,0.008066
PropCat_Outdoor,0.007238


## <font color='blue'>4.d) Forêt aléatoire </font>

### Première estimation

In [ ]:
rf_model = RandomForestClassifier(random_state=42)
y_pred_RFC= train_and_evaluate_model(rf_model, "Random Forest", X_train, y_train, X_val, y_val)
# y_pred_RFC = rf_model.predict(X_val)
# evaluate_model(y_val, y_pred_RFC, "Random Forest")


Random Forest
Overall accuracy : 0.1479
min precision:  0.0
median precision :  0.0865
max precision :  0.3391
min recall :  0.0
median recall :  0.1037
max recall :  0.6307
min fscore :  0.0
median fscore :  0.0935
max fscore :  0.441


C:\Users\ADUBOIS\AppData\Local\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [ ]:
feat_imp = pd.DataFrame({'importance': rf_model.feature_importances_}, index=rf_model.feature_names_in_)
feat_imp.sort_values(by=['importance'], ascending=False).head(10)

,importance
distStd,0.899701
Stat_resp_rep,0.018789
ratioStd,0.010766
Bor_inc_rep,0.008409
Bor_resp_rep,0.008006
H26,0.006538
H1117,0.006364
inner,0.003326
PropCat_Non Residential,0.002979
PropCat_Dwelling,0.002882


### Recherche des hyper paramètres

La recherche d'hyper paramètres sur le jeu de données réduit améliore légèrement l'accuracy. Cependent, comme elle reste inférieure à 20%, nous n'avons pas appliqué ce modèle au jeu de données complet.

In [ ]:
param_distributions = {
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [None, 10, 20, 30, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False],
}

model = RandomForestClassifier(random_state=42)
y_pred_RFC_opt, best_params_RFC = find_best_params_and_predict(model, param_distributions, X_train_r, y_train_r, X_val_r, y_val_r, model_name="Random Forest", n_iter=50)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


C:\Users\ADUBOIS\AppData\Local\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:909: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().




Random Forest - Best Parameters: {'n_estimators': 100, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_depth': 10, 'bootstrap': False}
Random Forest - Best Cross-Validation Score: 0.3978

Random Forest
Overall accuracy : 0.1619
min precision:  0.0
median precision :  0.0829
max precision :  0.312
min recall :  0.0
median recall :  0.0593
max recall :  0.7678
min fscore :  0.0
median fscore :  0.0484
max fscore :  0.4437


C:\Users\ADUBOIS\AppData\Local\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



## <font color='blue'>4.e) Gradient Boosting </font>

### Première estimation

In [ ]:
gbm_model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
y_pred_GBC= train_and_evaluate_model(gbm_model, "Gradient Boosting", X_train, y_train, X_val, y_val)



Gradient Boosting
Overall accuracy : 0.1542
min precision:  0.0
median precision :  0.0569
max precision :  0.3552
min recall :  0.0
median recall :  0.0515
max recall :  0.7395
min fscore :  0.0
median fscore :  0.045
max fscore :  0.4799


C:\Users\ADUBOIS\AppData\Local\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [ ]:
feat_imp = pd.DataFrame({'importance': gbm_model.feature_importances_}, index=gbm_model.feature_names_in_)
feat_imp.sort_values(by=['importance'], ascending=False).head(10)

,importance
distStd,0.889365
H26,0.023906
Stat_resp_rep,0.011176
ratioStd,0.010252
Borough_E09000033,0.008374
Borough_E09000022,0.005225
H1117,0.005147
PropCat_Dwelling,0.004979
Bor_inc_rep,0.004715
PropCat_Other Residential,0.004526


### Recherche des hyper paramètres

In [ ]:
gbm_param_dist = {
    'learning_rate': np.logspace(-3, 0, 10), # % contribution de chaque apprenant faible
    'n_estimators': np.arange(100, 500, 50), # nd d'arbres
    'subsample': np.linspace(0.5, 1, 5),   # % de lignes utilisées pour la construction de chaque arbre
    'max_depth': np.arange(3, 15, 2), # profondeur maximale des arbres
    'min_samples_split' : np.linspace(0.001, 0.05, 5), # % d'observations utilisé pour créer un noeud dans l'arbre
    'max_features' : [25,38, None] # nb de variables explicatives à considérer pour créer un noeud
}

model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
y_pred_XGB_opt, best_params_XGB = find_best_params_and_predict(model, gbm_param_dist, X_train_r, y_train_r, X_val_r, y_val_r, model_name="Gradient Boosting", n_iter=50)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


C:\Users\ADUBOIS\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:437: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().




Gradient Boosting - Best Parameters: {'subsample': 0.75, 'n_estimators': 250, 'min_samples_split': 0.001, 'max_features': 38, 'max_depth': 11, 'learning_rate': 0.01}
Gradient Boosting - Best Cross-Validation Score: 0.4070

Gradient Boosting
Overall accuracy : 0.1545
min precision:  0.0
median precision :  0.0665
max precision :  0.3699
min recall :  0.0
median recall :  0.0596
max recall :  0.7183
min fscore :  0.0
median fscore :  0.0534
max fscore :  0.4883


C:\Users\ADUBOIS\AppData\Local\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



## <font color='blue'>4.f) XG Boost </font>

### Première estimation

In [ ]:
xgb_model = XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42, use_label_encoder=False)
y_pred_XGBC= train_and_evaluate_model(xgb_model, "XGBoost", X_train, y_train, X_val, y_val)


C:\Users\ADUBOIS\AppData\Local\anaconda3\lib\site-packages\xgboost\core.py:158: UserWarning:

[13:38:08] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.





XGBoost
Overall accuracy : 0.155
min precision:  0.0
median precision :  0.0587
max precision :  0.3493
min recall :  0.0
median recall :  0.0509
max recall :  0.7434
min fscore :  0.0
median fscore :  0.0437
max fscore :  0.4752


C:\Users\ADUBOIS\AppData\Local\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [ ]:
feat_imp = pd.DataFrame({'importance': xgb_model.feature_importances_}, index=xgb_model.feature_names_in_)
feat_imp.sort_values(by=['importance'], ascending=False).head(10)

,importance
distStd,0.298282
H26,0.042794
Borough_E09000033,0.034095
Borough_E09000022,0.030976
Stat_resp_rep,0.029465
inner,0.023734
PropCat_Other Residential,0.023095
PropCat_Outdoor,0.022441
Borough_E09000012,0.020068
Borough_E09000010,0.019891


### Recherche des hyper paramètres

In [ ]:
xgb_param_dist = {
    'n_estimators': np.arange(100, 500, 50),
    'max_depth': np.arange(3, 15, 2),
    'learning_rate': np.logspace(-3, 0, 10),
    'subsample': np.linspace(0.5, 1, 5),
    'colsample_bytree': np.linspace(0.5, 1, 5),
    'gamma': np.linspace(0, 1, 5),
    'reg_alpha': np.logspace(-3, 1, 5),
    'reg_lambda': np.logspace(-3, 1, 5)
}

model = XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42, use_label_encoder=False)
y_pred_XGB_opt, best_params_XGB = find_best_params_and_predict(model, xgb_param_dist, X_train_r, y_train_r, X_val_r, y_val_r, model_name="XGB", n_iter=50)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


C:\Users\ADUBOIS\AppData\Local\anaconda3\lib\site-packages\xgboost\core.py:158: UserWarning:

[10:30:30] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.





XGB - Best Parameters: {'subsample': 0.875, 'reg_lambda': 0.1, 'reg_alpha': 0.01, 'n_estimators': 300, 'max_depth': 9, 'learning_rate': 0.046415888336127774, 'gamma': 1.0, 'colsample_bytree': 0.5}
XGB - Best Cross-Validation Score: 0.4052

XGB
Overall accuracy : 0.1543
min precision:  0.0
median precision :  0.0624
max precision :  0.3663
min recall :  0.0
median recall :  0.0608
max recall :  0.7203
min fscore :  0.0
median fscore :  0.0527
max fscore :  0.4856


C:\Users\ADUBOIS\AppData\Local\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



## <font color='blue'>4.g) Light Gradient Boosting </font>

### Première estimation

In [ ]:
lgbm_model = LGBMClassifier(n_estimators=100, learning_rate=0.1, max_depth=-1, random_state=42)
y_pred_LGBMC= train_and_evaluate_model(lgbm_model, "LightGBM", X_train, y_train, X_val, y_val)

C:\Users\ADUBOIS\AppData\Local\anaconda3\lib\site-packages\joblib\externals\loky\backend\context.py:150: UserWarning:

Could not find the number of physical cores for the following reason:
[WinError 2] Le fichier spécifié est introuvable
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.

  File "C:\Users\ADUBOIS\AppData\Local\anaconda3\lib\site-packages\joblib\externals\loky\backend\context.py", line 227, in _count_physical_cores
    cpu_info = subprocess.run(
  File "C:\Users\ADUBOIS\AppData\Local\anaconda3\lib\subprocess.py", line 503, in run
    with Popen(*popenargs, **kwargs) as process:
  File "C:\Users\ADUBOIS\AppData\Local\anaconda3\lib\subprocess.py", line 971, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "C:\Users\ADUBOIS\AppData\Local\anaconda3\lib\subprocess.py", line 1440, in _execute_child
    hp, ht, pid, tid = _winapi.CreateProcess(execu

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.041894 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 383
[LightGBM] [Info] Number of data points in the train set: 726399, number of used features: 50
[LightGBM] [Info] Start training from score -1.751207
[LightGBM] [Info] Start training from score -1.844272
[LightGBM] [Info] Start training from score -1.729645
[LightGBM] [Info] Start training from score -1.611069
[LightGBM] [Info] Start training from score -2.071138
[LightGBM] [Info] Start training from score -1.800408

LightGBM
Overall accuracy : 0.1542
min precision:  0.0
median precision :  0.057
max precision :  0.3556
min recall :  0.0
median recall :  0.0523
max recall :  0.7405
min fscore :  0.0
median fscore :  0.0451
max fscore :  0.4804


C:\Users\ADUBOIS\AppData\Local\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [ ]:
feat_imp = pd.DataFrame({'importance': lgbm_model.feature_importances_}, index=lgbm_model.feature_names_in_)
feat_imp.sort_values(by=['importance'], ascending=False).head(10)

,importance
distStd,6847
ratioStd,996
H26,866
Bor_inc_rep,814
Stat_resp_rep,638
H1117,604
PropCat_Dwelling,599
PropCat_Other_Residential,492
PropCat_Non_Residential,491
Borough_E09000033,441


### Recherche des hyper paramètres

In [ ]:
xgb_param_dist = {
    'n_estimators': np.arange(100, 500, 50),
    'max_depth': np.arange(3, 15, 2),
    'learning_rate': np.logspace(-3, 0, 10),
    'subsample': np.linspace(0.5, 1, 5),
    'colsample_bytree': np.linspace(0.5, 1, 5),
    'gamma': np.linspace(0, 1, 5),
    'reg_alpha': np.logspace(-3, 1, 5),
    'reg_lambda': np.logspace(-3, 1, 5)
}

model = XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42, use_label_encoder=False)
y_pred_XGB_opt, best_params_XGB = find_best_params_and_predict(model, xgb_param_dist, X_train_r, y_train_r, X_val_r, y_val_r, model_name="XGB", n_iter=50)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


C:\Users\ADUBOIS\AppData\Local\anaconda3\lib\site-packages\xgboost\core.py:158: UserWarning:

[11:01:50] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.





XGB - Best Parameters: {'subsample': 0.875, 'reg_lambda': 0.1, 'reg_alpha': 0.01, 'n_estimators': 300, 'max_depth': 9, 'learning_rate': 0.046415888336127774, 'gamma': 1.0, 'colsample_bytree': 0.5}
XGB - Best Cross-Validation Score: 0.4052

XGB
Overall accuracy : 0.1543
min precision:  0.0
median precision :  0.0624
max precision :  0.3663
min recall :  0.0
median recall :  0.0608
max recall :  0.7203
min fscore :  0.0
median fscore :  0.0527
max fscore :  0.4856


C:\Users\ADUBOIS\AppData\Local\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



# <font color='red'> 6) Conclusion
 </font>


Etonnament, les résultats de modélisation du temps catégorisé en 6 classes (`ResponseTimeCategory2`) sont plus mauvais que ceux du temps catégorisé en 10 classes (`ResponseTimeCategory`). L'optimisation des hyper-paramètres n'a que très peu amélioré l'accuracy.

Dans le prochain notebook, nous modélisons la variable binaire `ResponseTimeBinary` qui indique si le temps de réponse est inférieur (ou non) à 6 minutes.